In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_447820/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/17 17:08:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/17 17:08:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/17 17:08:48 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data
data = {
    'player_id': [1, 1, 2, 3, 3],
    'device_id': [2, 2, 3, 1, 4],
    'event_date': ['2016-03-01', '2016-03-02', '2017-06-25', '2016-03-01', '2016-07-03'],
    'games_played': [5, 6, 1, 0, 5]
}

# Create DataFrame
activity_df = pd.DataFrame(data)

# Convert event_date to datetime
activity_df['event_date'] = pd.to_datetime(activity_df['event_date'])

# Display the result
print(activity_df)




df_person = spark.createDataFrame(activity_df)
df_person.createOrReplaceTempView("Activity")



   player_id  device_id event_date  games_played
0          1          2 2016-03-01             5
1          1          2 2016-03-02             6
2          2          3 2017-06-25             1
3          3          1 2016-03-01             0
4          3          4 2016-07-03             5


In [4]:
query = """
    select distinct event_date
    from Activity
    order by event_date asc
"""

In [8]:
spark.sql(query).show()

+-------------------+
|         event_date|
+-------------------+
|2016-03-01 00:00:00|
|2016-03-02 00:00:00|
|2016-07-03 00:00:00|
|2017-06-25 00:00:00|
+-------------------+



In [5]:
query_1 = """
    select
        *,
        row_number() over (partition by player_id order by event_date asc) as order_login,
        date_add(event_date, 1) = lead(event_date, 1, event_date) over (partition by player_id order by event_date asc) as day_one
    from
        Activity a 
"""

In [34]:
spark.sql(query_1).show()

+---------+---------+-------------------+------------+-----------+-------+
|player_id|device_id|         event_date|games_played|order_login|day_one|
+---------+---------+-------------------+------------+-----------+-------+
|        1|        2|2016-03-01 00:00:00|           5|          1|   true|
|        1|        2|2016-03-02 00:00:00|           6|          2|  false|
|        2|        3|2017-06-25 00:00:00|           1|          1|  false|
|        3|        1|2016-03-01 00:00:00|           0|          1|  false|
|        3|        4|2016-07-03 00:00:00|           5|          2|  false|
+---------+---------+-------------------+------------+-----------+-------+



In [ ]:
+---------+---------+-------------------+------------+
|player_id|device_id|         event_date|games_played|
+---------+---------+-------------------+------------+
|        1|        2|2016-03-01 00:00:00|           5|
|        1|        2|2016-03-02 00:00:00|           6|
|        2|        3|2017-06-25 00:00:00|           1|
|        3|        1|2016-03-01 00:00:00|           0|
|        3|        4|2016-07-03 00:00:00|           5|
+---------+---------+-------------------+------------+

In [6]:
query_2 = f"""
    select 
        *, 
        sum(if(day_one=true, 1, 0)) over(partition by event_date) / count(*) over(partition by event_date) as Day1_retention,
        sum(if(day_one=true, 1, 0)) over(partition by event_date) as sum_day_one,
        count(*) over(partition by event_date) as count_all
    from ({query_1}) 
    where order_login = 1

"""

In [32]:
spark.sql(query_2).show()

+---------+---------+-------------------+------------+-----------+-------+--------------+-----------+---------+
|player_id|device_id|         event_date|games_played|order_login|day_one|Day1_retention|sum_day_one|count_all|
+---------+---------+-------------------+------------+-----------+-------+--------------+-----------+---------+
|        1|        2|2016-03-01 00:00:00|           5|          1|  false|           0.0|          0|        2|
|        3|        1|2016-03-01 00:00:00|           0|          1|  false|           0.0|          0|        2|
|        2|        3|2017-06-25 00:00:00|           1|          1|  false|           0.0|          0|        1|
+---------+---------+-------------------+------------+-----------+-------+--------------+-----------+---------+



In [13]:
query_3 = f"""
    with t as (
        select
            *,
            row_number() over (partition by player_id order by event_date asc) as order_login,
            date_add(event_date, 1) = lead(event_date, 1, event_date) over (partition by player_id order by event_date asc) as day_one,
            lead(event_date, 1, event_date) over (partition by player_id order by event_date asc) as abc
        from
            Activity a  
    )

    select 
        *
    from t
    
"""

In [14]:
spark.sql(query_3).show()
spark.sql(query_3).explain()

+---------+---------+-------------------+------------+-----------+-------+-------------------+
|player_id|device_id|         event_date|games_played|order_login|day_one|                abc|
+---------+---------+-------------------+------------+-----------+-------+-------------------+
|        1|        2|2016-03-01 00:00:00|           5|          1|   true|2016-03-02 00:00:00|
|        1|        2|2016-03-02 00:00:00|           6|          2|  false|2016-03-02 00:00:00|
|        2|        3|2017-06-25 00:00:00|           1|          1|  false|2017-06-25 00:00:00|
|        3|        1|2016-03-01 00:00:00|           0|          1|  false|2016-07-03 00:00:00|
|        3|        4|2016-07-03 00:00:00|           5|          2|  false|2016-07-03 00:00:00|
+---------+---------+-------------------+------------+-----------+-------+-------------------+

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [player_id#0L, device_id#1L, event_date#2, games_played#3L, order_login#192, 

In [101]:
query_3_1 = """
    with t as (
        select
            *,
            row_number() over (partition by player_id order by event_date asc) as order_login,
            lead(event_date) over (partition by player_id order by event_date asc) as day_lead
        from
            Activity a  
    )

    select 
        *
    from t
"""

In [102]:
spark.sql(query_3_1).show()
spark.sql(query_3_1).explain()

+---------+---------+-------------------+------------+-----------+-------------------+
|player_id|device_id|         event_date|games_played|order_login|           day_lead|
+---------+---------+-------------------+------------+-----------+-------------------+
|        1|        2|2016-03-01 00:00:00|           5|          1|2016-03-02 00:00:00|
|        1|        2|2016-03-02 00:00:00|           6|          2|               NULL|
|        2|        3|2017-06-25 00:00:00|           1|          1|               NULL|
|        3|        1|2016-03-01 00:00:00|           0|          1|2016-07-03 00:00:00|
|        3|        4|2016-07-03 00:00:00|           5|          2|               NULL|
+---------+---------+-------------------+------------+-----------+-------------------+

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Window [row_number() windowspecdefinition(player_id#0L, event_date#2 ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) A

In [116]:
query_3_2 = """

    with t as (
        select
            *,
            row_number() over (partition by player_id order by event_date asc) as order_login,
            lead(event_date) over (partition by player_id order by event_date asc) as day_lead
        from
            Activity a  
    )

    select 
        *
    from t
    where order_login = 1
"""

In [117]:
spark.sql(query_3_2).show(truncate=False)
spark.sql(query_3_2).explain(extended=True)

+---------+---------+-------------------+------------+-----------+--------+
|player_id|device_id|event_date         |games_played|order_login|day_lead|
+---------+---------+-------------------+------------+-----------+--------+
|1        |2        |2016-03-01 00:00:00|5           |1          |NULL    |
|2        |3        |2017-06-25 00:00:00|1           |1          |NULL    |
|3        |1        |2016-03-01 00:00:00|0           |1          |NULL    |
+---------+---------+-------------------+------------+-----------+--------+

== Parsed Logical Plan ==
CTE [t]
:  +- 'SubqueryAlias t
:     +- 'Project [*, 'row_number() windowspecdefinition('player_id, 'event_date ASC NULLS FIRST, unspecifiedframe$()) AS order_login#2035, 'lead('event_date) windowspecdefinition('player_id, 'event_date ASC NULLS FIRST, unspecifiedframe$()) AS day_lead#2036]
:        +- 'SubqueryAlias a
:           +- 'UnresolvedRelation [Activity], [], false
+- 'Project [*]
   +- 'Filter ('order_login = 1)
      +- 'Unres

In [115]:
print("""
== Parsed Logical Plan ==\nCTE [t]\n:  +- 'SubqueryAlias t\n:     +- 'Project [*, 'row_number() windowspecdefinition('player_id, 'event_date ASC NULLS FIRST, unspecifiedframe$()) AS order_login#1988, 'lead('event_date) windowspecdefinition('player_id, 'event_date ASC NULLS FIRST, unspecifiedframe$()) AS day_lead#1989]\n:        +- 'SubqueryAlias a\n:           +- 'UnresolvedRelation [Activity], [], false\n+- 'Project [*]\n   +- 'Filter ('order_login = 1)\n      +- 'UnresolvedRelation [t], [], false\n\n== Analyzed Logical Plan ==\nplayer_id: bigint, device_id: bigint, event_date: timestamp, games_played: bigint, order_login: int, day_lead: timestamp\nWithCTE\n:- CTERelationDef 56, false\n:  +- SubqueryAlias t\n:     +- Project [player_id#0L, device_id#1L, event_date#2, games_played#3L, order_login#1988, day_lead#1989]\n:        +- Project [player_id#0L, device_id#1L, event_date#2, games_played#3L, order_login#1988, day_lead#1989, order_login#1988, day_lead#1989]\n:           +- Window [row_number() windowspecdefinition(player_id#0L, event_date#2 ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS order_login#1988, lead(event_date#2, 1, null) windowspecdefinition(player_id#0L, event_date#2 ASC NULLS FIRST, specifiedwindowframe(RowFrame, 1, 1)) AS day_lead#1989], [player_id#0L], [event_date#2 ASC NULLS FIRST]\n:              +- Project [player_id#0L, device_id#1L, event_date#2, games_played#3L]\n:                 +- SubqueryAlias a\n:                    +- SubqueryAlias activity\n:                       +- View (`Activity`, [player_id#0L,device_id#1L,event_date#2,games_played#3L])\n:                          +- LogicalRDD [player_id#0L, device_id#1L, event_date#2, games_played#3L], false\n+- Project [player_id#0L, device_id#1L, event_date#2, games_played#3L, order_login#1988, day_lead#1989]\n   +- Filter (order_login#1988 = 1)\n      +- SubqueryAlias t\n         +- CTERelationRef 56, true, [player_id#0L, device_id#1L, event_date#2, games_played#3L, order_login#1988, day_lead#1989]\n\n== Optimized Logical Plan ==\nFilter (order_login#1988 = 1)\n+- Window [row_number() windowspecdefinition(player_id#0L, event_date#2 ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS order_login#1988, lead(event_date#2, 1, null) windowspecdefinition(player_id#0L, event_date#2 ASC NULLS FIRST, specifiedwindowframe(RowFrame, 1, 1)) AS day_lead#1989], [player_id#0L], [event_date#2 ASC NULLS FIRST]\n   +- WindowGroupLimit [player_id#0L], [event_date#2 ASC NULLS FIRST], row_number(), 1\n      +- LogicalRDD [player_id#0L, device_id#1L, event_date#2, games_played#3L], false\n\n== Physical Plan ==\nAdaptiveSparkPlan isFinalPlan=false\n+- Filter (order_login#1988 = 1)\n   +- Window [row_number() windowspecdefinition(player_id#0L, event_date#2 ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS order_login#1988, lead(event_date#2, 1, null) windowspecdefinition(player_id#0L, event_date#2 ASC NULLS FIRST, specifiedwindowframe(RowFrame, 1, 1)) AS day_lead#1989], [player_id#0L], [event_date#2 ASC NULLS FIRST]\n      +- WindowGroupLimit [player_id#0L], [event_date#2 ASC NULLS FIRST], row_number(), 1, Final\n         +- Sort [player_id#0L ASC NULLS FIRST, event_date#2 ASC NULLS FIRST], false, 0\n            +- Exchange hashpartitioning(player_id#0L, 200), ENSURE_REQUIREMENTS, [plan_id=4648]\n               +- WindowGroupLimit [player_id#0L], [event_date#2 ASC NULLS FIRST], row_number(), 1, Partial\n                  +- Sort [player_id#0L ASC NULLS FIRST, event_date#2 ASC NULLS FIRST], false, 0\n                     +- Scan ExistingRDD[player_id#0L,device_id#1L,event_date#2,games_played#3L]\n|

""")


== Parsed Logical Plan ==
CTE [t]
:  +- 'SubqueryAlias t
:     +- 'Project [*, 'row_number() windowspecdefinition('player_id, 'event_date ASC NULLS FIRST, unspecifiedframe$()) AS order_login#1988, 'lead('event_date) windowspecdefinition('player_id, 'event_date ASC NULLS FIRST, unspecifiedframe$()) AS day_lead#1989]
:        +- 'SubqueryAlias a
:           +- 'UnresolvedRelation [Activity], [], false
+- 'Project [*]
   +- 'Filter ('order_login = 1)
      +- 'UnresolvedRelation [t], [], false

== Analyzed Logical Plan ==
player_id: bigint, device_id: bigint, event_date: timestamp, games_played: bigint, order_login: int, day_lead: timestamp
WithCTE
:- CTERelationDef 56, false
:  +- SubqueryAlias t
:     +- Project [player_id#0L, device_id#1L, event_date#2, games_played#3L, order_login#1988, day_lead#1989]
:        +- Project [player_id#0L, device_id#1L, event_date#2, games_played#3L, order_login#1988, day_lead#1989, order_login#1988, day_lead#1989]
:           +- Window [row_number() win

In [95]:
query_3_3 = """
    with t as (
        select
            *,
            row_number() over (partition by player_id order by event_date asc) as order_login
        from
            Activity a  
    )

    select 
        *
    from t
"""

In [96]:
spark.sql(query_3_3).show()
spark.sql(query_3_3).explain()

+---------+---------+-------------------+------------+-----------+
|player_id|device_id|         event_date|games_played|order_login|
+---------+---------+-------------------+------------+-----------+
|        1|        2|2016-03-01 00:00:00|           5|          1|
|        1|        2|2016-03-02 00:00:00|           6|          2|
|        2|        3|2017-06-25 00:00:00|           1|          1|
|        3|        1|2016-03-01 00:00:00|           0|          1|
|        3|        4|2016-07-03 00:00:00|           5|          2|
+---------+---------+-------------------+------------+-----------+

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Window [row_number() windowspecdefinition(player_id#0L, event_date#2 ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS order_login#1735], [player_id#0L], [event_date#2 ASC NULLS FIRST]
   +- Sort [player_id#0L ASC NULLS FIRST, event_date#2 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitio

In [105]:
query_4 = """
    select
        *,
        row_number() over (partition by player_id order by event_date asc) as order_login
    from Activity a
"""

In [106]:
spark.sql(query_4).show()

+---------+---------+-------------------+------------+-----------+
|player_id|device_id|         event_date|games_played|order_login|
+---------+---------+-------------------+------------+-----------+
|        1|        2|2016-03-01 00:00:00|           5|          1|
|        1|        2|2016-03-02 00:00:00|           6|          2|
|        2|        3|2017-06-25 00:00:00|           1|          1|
|        3|        1|2016-03-01 00:00:00|           0|          1|
|        3|        4|2016-07-03 00:00:00|           5|          2|
+---------+---------+-------------------+------------+-----------+



In [107]:
query_5 = """
    select
        *
    from (
        select
            *,
            row_number() over (partition by player_id order by event_date asc) as order_login
        from Activity a
    ) where order_login = 1
"""

In [109]:
spark.sql(query_5).show()
spark.sql(query_5).explain()

+---------+---------+-------------------+------------+-----------+
|player_id|device_id|         event_date|games_played|order_login|
+---------+---------+-------------------+------------+-----------+
|        1|        2|2016-03-01 00:00:00|           5|          1|
|        2|        3|2017-06-25 00:00:00|           1|          1|
|        3|        1|2016-03-01 00:00:00|           0|          1|
+---------+---------+-------------------+------------+-----------+

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Filter (order_login#1955 = 1)
   +- Window [row_number() windowspecdefinition(player_id#0L, event_date#2 ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS order_login#1955], [player_id#0L], [event_date#2 ASC NULLS FIRST]
      +- WindowGroupLimit [player_id#0L], [event_date#2 ASC NULLS FIRST], row_number(), 1, Final
         +- Sort [player_id#0L ASC NULLS FIRST, event_date#2 ASC NULLS FIRST], false, 0
            +- Exchange h

In [119]:
query_6 = """
WITH t AS (
    SELECT
        *,
        row_number() OVER (PARTITION BY player_id ORDER BY event_date ASC) AS order_login,
        lead(event_date) OVER (PARTITION BY player_id ORDER BY event_date ASC) AS day_lead
    FROM
        Activity
)

SELECT
    player_id,
    device_id,
    event_date,
    games_played,
    CASE 
        WHEN order_login = 1 THEN LEAD(event_date) OVER (PARTITION BY player_id ORDER BY event_date ASC) 
        ELSE NULL 
    END AS day_lead
FROM
    t
WHERE
    order_login = 1

"""

In [120]:
spark.sql(query_6).show()

+---------+---------+-------------------+------------+--------+
|player_id|device_id|         event_date|games_played|day_lead|
+---------+---------+-------------------+------------+--------+
|        1|        2|2016-03-01 00:00:00|           5|    NULL|
|        2|        3|2017-06-25 00:00:00|           1|    NULL|
|        3|        1|2016-03-01 00:00:00|           0|    NULL|
+---------+---------+-------------------+------------+--------+

